<a href="https://colab.research.google.com/github/cuongko/Capstone-Project-Notebook/blob/main/Explore_Toronto_Canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=center><font size = 7>Segmenting and Clustering Neighborhoods in New York City</font></h1>

# **List of postal codes of Toronto Canada**






This part of the notebook use Beutiful Soup to scrap data from web to a dataframe then merge with geo data to creat new data frame to store Neighborhood, Longitude, Latitude and some information of Canada

## Scrap data from web

In [1]:
# intall packages need
!pip3 install requests
!pip3 install beautifulsoup4

In [2]:
# Establish environment
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# Load page from URL
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
# Print to see page content, use pprint for better display
# This too long and we can pass this step
# import pprint
# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(page.content)

In [5]:
# use table to get data that we need
venues = soup.find('table')
venues.prettify()

'<table class="wikitable sortable">\n <tbody>\n  <tr>\n   <th>\n    Postal Code\n   </th>\n   <th>\n    Borough\n   </th>\n   <th>\n    Neighbourhood\n   </th>\n  </tr>\n  <tr>\n   <td>\n    M1A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M2A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M3A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Parkwoods\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M4A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Victoria Village\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M5A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>\n    Regent Park, Harbourfront\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M6A\n   </td>\n   <td>\n    North York\n   </td>\n   <td>\n    Lawrence Manor, Lawrence Heights\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M7A\n   </td>\n   <td>\n    Downtown Toronto\n   </td>\n   <td>

In [6]:
# We found that, all values we need standing after <td> mark
# we load them into a list
toronto = venues.find_all('td')

In [7]:
print(toronto[0].text.strip(), ' ', # item has order % 3 = 0 is Postal Code
      toronto[1].text.strip(), ' ', # item has order % 3 = 1 is Borough
      toronto[2].text.strip())      # item has order % 3 = 2 is Neighbourhood  

M1A   Not assigned   Not assigned


In [8]:
# extract values into lists and creat dataframe from these lists
postal_code = []
borough = []
neighbourhood = []
for i in range(0, len(toronto)):
  if i % 3 == 0:
    postal_code.append(toronto[i].text.strip())
  elif i % 3 == 1:
    borough.append(toronto[i].text.strip())
  else:
    neighbourhood.append(toronto[i].text.strip())

# creat data frame
df = pd.DataFrame(list(zip(postal_code, borough,neighbourhood)),
                  columns = ['Postal Code', 'Borough', 'Neighbourhood'])

df.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
# creat final data frame by filter value != 'Not assigned'
df = df[df['Borough'] != 'Not assigned']
df.head(5)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df.shape

(103, 3)

## Add Latitude and Longitude to data frame

In [11]:
lng_lat = pd.read_csv('http://cocl.us/Geospatial_data')
lng_lat.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
toronto_venues = pd.merge(df, lng_lat, on = 'Postal Code', how = 'inner')
toronto_venues.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [13]:
toronto_venues.shape

(103, 5)

# Cluster Neighborhoods

Load data from 4square and Use k-means to cluster the neighborhood into 5 clusters

## Define Foursquare Credentials and Version

In [14]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

In [15]:
CLIENT_ID = '2C22B3K5HPKL43NSDAT3ZCM40VETWBHCSRAPCVEP35PQXU4O' # your Foursquare ID
CLIENT_SECRET = '3VEKLN2SCWLAX4VLGQUWDG3PBCMJCBCTRAKM1BAAX0VFENVH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2C22B3K5HPKL43NSDAT3ZCM40VETWBHCSRAPCVEP35PQXU4O
CLIENT_SECRET:3VEKLN2SCWLAX4VLGQUWDG3PBCMJCBCTRAKM1BAAX0VFENVH


## Filter dataframe contain Toronto and explore the first neighborhood

In [16]:
def contain(x):
  if 'Toronto' in x:
    return True

# Select borough contain Toronto
toronto_neighbour = toronto_venues[toronto_venues['Borough'].apply(contain) == True]

In [17]:
toronto_neighbour.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [18]:
neighborhood_latitude = toronto_neighbour.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_neighbour.loc[2, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto_neighbour.loc[2, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Get the Top 100 venues and in radius 500 m

In [19]:
# creat URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
                                                                                                                            CLIENT_SECRET, 
                                                                                                                            VERSION, 
                                                                                                                            neighborhood_latitude, 
                                                                                                                            neighborhood_longitude, 
                                                                                                                            500, # the radius 
                                                                                                                            100) # number of venue
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2C22B3K5HPKL43NSDAT3ZCM40VETWBHCSRAPCVEP35PQXU4O&client_secret=3VEKLN2SCWLAX4VLGQUWDG3PBCMJCBCTRAKM1BAAX0VFENVH&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fe45122f849016e716e4420'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 143,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [21]:
venues = results['response']['groups'][0]['items']

In [22]:
venues

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d16a941735',
     'name': 'Bakery',
     'pluralName': 'Bakeries',
     'primary': True,
     'shortName': 'Bakery'}],
   'id': '54ea41ad498e9a11e9e13308',
   'location': {'address': '362 King St E',
    'cc': 'CA',
    'city': 'Toronto',
    'country': 'Canada',
    'crossStreet': 'Trinity St',
    'distance': 143,
    'formattedAddress': ['362 King St E (Trinity St)',
     'Toronto ON M5A 1K9',
     'Canada'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.653446723052674,
      'lng': -79.3620167174383}],
    'lat': 43.653446723052674,
    'lng': -79.3620167174383,
    'postalCode': 'M5A 1K9',
    'state': 'ON'},
   'name': '

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# get toroto neighborhood
toronto_venues = getNearbyVenues(names = nearby_venues['name'],
                                 latitudes = nearby_venues['lat'],
                                 longitudes = nearby_venues['lng'], radius=500)

Roselle Desserts
Tandem Coffee
Cooper Koo Family YMCA
Morning Glory Cafe
Body Blitz Spa East
Impact Kitchen
The Extension Room
The Distillery Historic District
Corktown Common
SOMA chocolatemaker
Distillery Sunday Market
Figs Breakfast & Lunch
Arvo
Rooster Coffee
Cacao 70
Sumach Espresso
Starbucks
Dominion Pub and Kitchen
Young Centre for the Performing Arts
Parliament Square Park
Underpass Park
Cluny Bistro & Boulangerie
The Sweet Escape Patisserie
Brick Street Bakery
Berkeley Church
The Yoga Lounge
Caffe Furbo
Dark Horse Espresso Bar
El Catrin
Alumnae Theatre
Wildly Delicious Cafe
Soulpepper Theatre
ODIN Cafe + Bar
John Fluevog Shoes
Mill St. Brew Pub
Ontario Spring Water Sake Company
The Aviary
Arta Gallery
The Abnormal Beauty Company
Vistek
The Beer Store
TD Canada Trust
Savoury Grounds
Residence & Conference Centre
The Healthy Road
GW General


In [28]:
toronto_venues.groupby('Neighborhood').count().head(5)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alumnae Theatre,40,40,40,40,40,40
Arta Gallery,43,43,43,43,43,43
Arvo,42,42,42,42,42,42
Berkeley Church,27,27,27,27,27,27
Body Blitz Spa East,36,36,36,36,36,36


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 76 uniques categories.


## Analyze Each Neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Animal Shelter,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Dealership,Bakery,Bank,Beer Store,Boat or Ferry,Boutique,Breakfast Spot,Brewery,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,German Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hotel,Intersection,Italian Restaurant,Karaoke Bar,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Moving Target,Museum,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Skating Rink,Spa,Spanish Restaurant,Sporting Goods Shop,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Yoga Studio
0,Roselle Desserts,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Roselle Desserts,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Roselle Desserts,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselle Desserts,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselle Desserts,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Animal Shelter,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Dealership,Bakery,Bank,Beer Store,Boat or Ferry,Boutique,Breakfast Spot,Brewery,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,German Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hotel,Intersection,Italian Restaurant,Karaoke Bar,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Moving Target,Museum,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Skating Rink,Spa,Spanish Restaurant,Sporting Goods Shop,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Yoga Studio
0,Alumnae Theatre,0.000000,0.000000,0.025000,0.0,0.0,0.0,0.050000,0.000000,0.000000,0.000000,0.0,0.050000,0.025000,0.0,0.025000,0.0,0.000000,0.0,0.175000,0.0,0.025000,0.025000,0.000000,0.025000,0.000000,0.000000,0.025000,0.025000,0.0,0.000000,0.000000,0.025000,0.025000,0.025000,0.0,0.000000,0.0,0.025000,0.025000,0.000000,0.000000,0.0,0.000000,0.0,0.075000,0.025,0.0,0.025000,0.0,0.000000,0.000000,0.0,0.0,0.025,0.025000,0.000000,0.025000,0.0,0.0,0.0,0.000000,0.025000,0.025000,0.0,0.000000,0.025000,0.025000,0.0,0.0,0.000000,0.000000,0.075000,0.000000,0.0,0.000000,0.025000
1,Arta Gallery,0.000000,0.023256,0.023256,0.0,0.0,0.0,0.069767,0.023256,0.000000,0.046512,0.0,0.023256,0.023256,0.0,0.046512,0.0,0.023256,0.0,0.139535,0.0,0.023256,0.023256,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256,0.0,0.000000,0.023256,0.000000,0.023256,0.000000,0.0,0.000000,0.0,0.023256,0.023256,0.000000,0.023256,0.0,0.023256,0.0,0.023256,0.000,0.0,0.023256,0.0,0.000000,0.023256,0.0,0.0,0.000,0.023256,0.023256,0.023256,0.0,0.0,0.0,0.023256,0.023256,0.000000,0.0,0.023256,0.000000,0.000000,0.0,0.0,0.023256,0.000000,0.069767,0.000000,0.0,0.023256,0.000000
2,Arvo,0.000000,0.023810,0.023810,0.0,0.0,0.0,0.071429,0.000000,0.000000,0.047619,0.0,0.023810,0.023810,0.0,0.047619,0.0,0.023810,0.0,0.142857,0.0,0.023810,0.023810,0.000000,0.000000,0.023810,0.000000,0.000000,0.023810,0.0,0.000000,0.023810,0.000000,0.023810,0.000000,0.0,0.000000,0.0,0.023810,0.023810,0.000000,0.023810,0.0,0.023810,0.0,0.023810,0.000,0.0,0.023810,0.0,0.000000,0.023810,0.0,0.0,0.000,0.023810,0.023810,0.023810,0.0,0.0,0.0,0.023810,0.023810,0.000000,0.0,0.023810,0.000000,0.000000,0.0,0.0,0.023810,0.000000,0.071429,0.000000,0.0,0.023810,0.000000
3,Berkeley Church,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.037037,0.000000,0.000000,0.000000,0.0,0.074074,0.000000,0.0,0.037037,0.0,0.000000,0.0,0.185185,0.0,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.037037,0.000000,0.0,0.037037,0.000000,0.037037,0.000000,0.037037,0.0,0.037037,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.111111,0.000,0.0,0.037037,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.037037,0.0,0.000000,0.037037,0.037037,0.0,0.0,0.000000,0.037037,0.037037,0.037037,0.0,0.000000,0.037037
4,Body Blitz Spa East,0.027778,0.000000,0.000000,0.0,0.0,0.0,0.083333,0.027778,0.027778,0.000000,0.0,0.055556,0.000000,0.0,0.055556,0.0,0.027778,0.0,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.027778,0.027778,0.027778,0.000000,0.0,0.000000,0.027778,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.027778,0.000000,0.027778,0.000000,0.0,0.027778,0.0,0.000000,0.000,0.0,0.000000,0.0,0.027778,0.027778,0.0,0.0,0.000,0.055556,0.027778,0.000000,0.0,0.0,0.0,0.083333,0.027778,0.000000,0.0,0.000000,0.000000,0.027778,0.0,0.0,0.000000,0.027778,0.055556,0.000000,0.0,0.000000,0.027778


In [32]:
toronto_grouped.shape

(46, 77)

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alumnae Theatre----
                venue  freq
0         Coffee Shop  0.18
1             Theater  0.08
2  Italian Restaurant  0.08
3      Breakfast Spot  0.05
4              Bakery  0.05


----Arta Gallery----
           venue  freq
0    Coffee Shop  0.14
1         Bakery  0.07
2        Theater  0.07
3  Boat or Ferry  0.05
4           Café  0.05


----Arvo----
           venue  freq
0    Coffee Shop  0.14
1         Bakery  0.07
2        Theater  0.07
3  Boat or Ferry  0.05
4           Café  0.05


----Berkeley Church----
                venue  freq
0         Coffee Shop  0.19
1  Italian Restaurant  0.11
2      Breakfast Spot  0.07
3         Yoga Studio  0.04
4        Skating Rink  0.04


----Body Blitz Spa East----
         venue  freq
0  Coffee Shop  0.11
1       Bakery  0.08
2          Pub  0.08
3      Theater  0.06
4         Park  0.06


----Brick Street Bakery----
         venue  freq
0  Coffee Shop  0.12
1      Theater  0.07
2       Bakery  0.07
3         Café  0.05
4    Pet 

Let's put those into a pandas frame

In [34]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alumnae Theatre,Coffee Shop,Theater,Italian Restaurant,Bakery,Breakfast Spot,Yoga Studio,Farmers Market,Karaoke Bar,Gym Pool,German Restaurant
1,Arta Gallery,Coffee Shop,Theater,Bakery,Café,Boat or Ferry,Hotel,Mexican Restaurant,Chocolate Shop,Liquor Store,Italian Restaurant
2,Arvo,Coffee Shop,Theater,Bakery,Café,Boat or Ferry,Hotel,Mexican Restaurant,Chocolate Shop,Liquor Store,Italian Restaurant
3,Berkeley Church,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Sandwich Place,Furniture / Home Store,Grocery Store,Food Truck,Event Space,Discount Store
4,Body Blitz Spa East,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,French Restaurant,Mexican Restaurant,Mediterranean Restaurant


In [36]:
toronto_grouped.head(5)

,Neighborhood,Animal Shelter,Antique Shop,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Dealership,Bakery,Bank,Beer Store,Boat or Ferry,Boutique,Breakfast Spot,Brewery,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,German Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hotel,Intersection,Italian Restaurant,Karaoke Bar,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Moving Target,Museum,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Skating Rink,Spa,Spanish Restaurant,Sporting Goods Shop,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Tram Station,Yoga Studio
0,Alumnae Theatre,0.000000,0.000000,0.025000,0.0,0.0,0.0,0.050000,0.000000,0.000000,0.000000,0.0,0.050000,0.025000,0.0,0.025000,0.0,0.000000,0.0,0.175000,0.0,0.025000,0.025000,0.000000,0.025000,0.000000,0.000000,0.025000,0.025000,0.0,0.000000,0.000000,0.025000,0.025000,0.025000,0.0,0.000000,0.0,0.025000,0.025000,0.000000,0.000000,0.0,0.000000,0.0,0.075000,0.025,0.0,0.025000,0.0,0.000000,0.000000,0.0,0.0,0.025,0.025000,0.000000,0.025000,0.0,0.0,0.0,0.000000,0.025000,0.025000,0.0,0.000000,0.025000,0.025000,0.0,0.0,0.000000,0.000000,0.075000,0.000000,0.0,0.000000,0.025000
1,Arta Gallery,0.000000,0.023256,0.023256,0.0,0.0,0.0,0.069767,0.023256,0.000000,0.046512,0.0,0.023256,0.023256,0.0,0.046512,0.0,0.023256,0.0,0.139535,0.0,0.023256,0.023256,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256,0.0,0.000000,0.023256,0.000000,0.023256,0.000000,0.0,0.000000,0.0,0.023256,0.023256,0.000000,0.023256,0.0,0.023256,0.0,0.023256,0.000,0.0,0.023256,0.0,0.000000,0.023256,0.0,0.0,0.000,0.023256,0.023256,0.023256,0.0,0.0,0.0,0.023256,0.023256,0.000000,0.0,0.023256,0.000000,0.000000,0.0,0.0,0.023256,0.000000,0.069767,0.000000,0.0,0.023256,0.000000
2,Arvo,0.000000,0.023810,0.023810,0.0,0.0,0.0,0.071429,0.000000,0.000000,0.047619,0.0,0.023810,0.023810,0.0,0.047619,0.0,0.023810,0.0,0.142857,0.0,0.023810,0.023810,0.000000,0.000000,0.023810,0.000000,0.000000,0.023810,0.0,0.000000,0.023810,0.000000,0.023810,0.000000,0.0,0.000000,0.0,0.023810,0.023810,0.000000,0.023810,0.0,0.023810,0.0,0.023810,0.000,0.0,0.023810,0.0,0.000000,0.023810,0.0,0.0,0.000,0.023810,0.023810,0.023810,0.0,0.0,0.0,0.023810,0.023810,0.000000,0.0,0.023810,0.000000,0.000000,0.0,0.0,0.023810,0.000000,0.071429,0.000000,0.0,0.023810,0.000000
3,Berkeley Church,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.037037,0.000000,0.000000,0.000000,0.0,0.074074,0.000000,0.0,0.037037,0.0,0.000000,0.0,0.185185,0.0,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.037037,0.000000,0.0,0.037037,0.000000,0.037037,0.000000,0.037037,0.0,0.037037,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.111111,0.000,0.0,0.037037,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.037037,0.0,0.000000,0.037037,0.037037,0.0,0.0,0.000000,0.037037,0.037037,0.037037,0.0,0.000000,0.037037
4,Body Blitz Spa East,0.027778,0.000000,0.000000,0.0,0.0,0.0,0.083333,0.027778,0.027778,0.000000,0.0,0.055556,0.000000,0.0,0.055556,0.0,0.027778,0.0,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.027778,0.027778,0.027778,0.000000,0.0,0.000000,0.027778,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.027778,0.000000,0.027778,0.000000,0.0,0.027778,0.0,0.000000,0.000,0.0,0.000000,0.0,0.027778,0.027778,0.0,0.0,0.000,0.055556,0.027778,0.000000,0.0,0.0,0.0,0.083333,0.027778,0.000000,0.0,0.000000,0.000000,0.027778,0.0,0.0,0.000000,0.027778,0.055556,0.000000,0.0,0.000000,0.027778


## Clustering Neighborhoods
Using k-means to cluster the neighborhood into 5 clusters and visualize to a folium map

### Clustering

In [37]:
kclusters = 5

toronto_clusters = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_clusters)

kmeans.labels_[0:10] 

array([2, 1, 1, 2, 4, 1, 1, 1, 4, 4], dtype=int32)

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [39]:
neighborhoods_venues_sorted.head(5)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Alumnae Theatre,Coffee Shop,Theater,Italian Restaurant,Bakery,Breakfast Spot,Yoga Studio,Farmers Market,Karaoke Bar,Gym Pool,German Restaurant
1,1,Arta Gallery,Coffee Shop,Theater,Bakery,Café,Boat or Ferry,Hotel,Mexican Restaurant,Chocolate Shop,Liquor Store,Italian Restaurant
2,1,Arvo,Coffee Shop,Theater,Bakery,Café,Boat or Ferry,Hotel,Mexican Restaurant,Chocolate Shop,Liquor Store,Italian Restaurant
3,2,Berkeley Church,Coffee Shop,Italian Restaurant,Breakfast Spot,Yoga Studio,Sandwich Place,Furniture / Home Store,Grocery Store,Food Truck,Event Space,Discount Store
4,4,Body Blitz Spa East,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,French Restaurant,Mexican Restaurant,Mediterranean Restaurant


In [40]:
toronto_venues.head(1)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Roselle Desserts,43.653447,-79.362017,Roselle Desserts,43.653447,-79.362017,Bakery


In [41]:
toronto_merged = pd.merge(neighborhoods_venues_sorted, toronto_venues, how = 'inner', on = 'Neighborhood' )

In [42]:
toronto_merged.head(1)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,2,Alumnae Theatre,Coffee Shop,Theater,Italian Restaurant,Bakery,Breakfast Spot,Yoga Studio,Farmers Market,Karaoke Bar,Gym Pool,German Restaurant,43.652756,-79.364753,Rooster Coffee,43.6519,-79.365609,Coffee Shop


### Visualize

In [43]:
!pip install geopy

In [44]:
from geopy.geocoders import Nominatim

In [45]:
address = 'Toronto City, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.65238435, -79.38356765.


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], 
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters